# Hurricane dorian

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import seaborn as sns
sns.reset_defaults()

In [ ]:
lat_c = 26.46  # Latitude of eye
lon_c = -95.93 # Longitude of eye

### Define equidistant projection around hurricane

In [ ]:
from satpy.scene import Scene
import satpy.writers
from pyresample.geometry import AreaDefinition

RESOLUTION = 2e3
EXTENT = 2e6

area_id = 'Dorian'
description = 'Projection centered on hurricane dorian'
proj_id = 'UK'
projection = {'proj': 'laea', 'lat_0': lat_c, 'lon_0': lon_c, 'a': 6371228.0, 'units': 'm'}
width = EXTENT / RESOLUTION
height = EXTENT / RESOLUTION
area_extent = (-EXTENT / 2, -EXTENT / 2, EXTENT / 2, EXTENT / 2)
area = AreaDefinition(area_id, description, proj_id, projection,
                      width, height, area_extent)

In [ ]:
lons, lats = area.get_lonlats()
lon_min, lon_max = lons.min(), lons.max()
lat_min, lat_max = lats.min(), lats.max()
indices = np.where(np.any(
    (l1c_data.latitude.data > lat_min) *
    (l1c_data.latitude.data < lat_max) * 
    (l1c_data.longitude.data > lon_min) *
    (l1c_data.longitude.data < lon_max),
    axis = -1
))[0]
scan_start = indices[0]
scan_end = indices[-1]

In [ ]:
import pandas as pd
from gprof_nn.data.l1c import L1CFile
l1c_file = "/pdata4/archive/GPM/1CR_GMI/1708/170825/1C-R.GPM.GMI.XCAL2016-C.20170825-S111924-E125156.019833.V05A.HDF5"
l1c_data = L1CFile(l1c_file).to_xarray_dataset()
n_scans = l1c_data.scans.size
n_pixels = l1c_data.pixels.size
d = (l1c_data.latitude - lat_c) ** 2 + (l1c_data.longitude - lon_c) ** 2
index = np.argmin(d.data)
i_c = index // n_pixels
start_time = l1c_data.scan_time[i_c]
start_time = pd.Timestamp(str(start_time.data)).to_pydatetime()
start_time

In [ ]:
from quantnn.qrnn import QRNN
model = QRNN.load("/gdata1/simon/gprof_nn/models/gprof_nn_3d_mhs_era5.pckl")

In [ ]:
from quantnn.qrnn import QRNN
model = QRNN.load("/gdata1/simon/gprof_nn/models/gprof_nn_1d_mhs_era5.pckl")

In [ ]:
model.sensor

## GOES-16

In [ ]:
from pathlib import Path
goes_files = list(Path("GOES-16/GOES-16-ABI-L1b-RadC/").glob("*s20172371132*"))

In [ ]:
from pathlib import Path
import pandas as pd
from datetime import timedelta
from pansat.products.satellite.goes import GOES16L1BRadiances
from satpy import Scene
from PIL import Image
if not Path("background.png").exists():
    #goes = GOES16L1BRadiances("C", list(range(1, 17)))
    #time = start_time
    #start_time = time - timedelta(minutes=time.minute % 10 + 5)
    #end_time = time - timedelta(minutes=time.minute % 10 - 5)
    #goes_files = goes.download(start_time, end_time)

    scene = Scene([str(f) for f in goes_files], reader="abi_l1b")
                 
    scene.load(["C13"])
    scene_r = scene.resample(
        area,
        radius_of_influence=10e3,
    )
    scene_r.save_dataset("C13", "background.png")
background = Image.open("background.png")

## Combined data

In [ ]:
from gprof_nn.data.combined import GPMCMBFile
from pyresample import geometry, kd_tree
cmb_file = "/pdata4/archive/GPM/2B_DPRGMI/1708/170825/2B.GPM.DPRGMI.CORRA2018.20170825-S111924-E125156.019833.V06A.HDF5"
data_cmb  = GPMCMBFile(cmb_file).to_xarray_dataset(smooth=True, profiles=False)
swath_l1c = geometry.SwathDefinition(
    lats=l1c_data.latitude.data[scan_start:scan_end],
    lons=l1c_data.longitude.data[scan_start:scan_end]
)
swath_cmb = geometry.SwathDefinition(
    lats=data_cmb.latitude.data,
    lons=data_cmb.longitude.data
)
sp_cmb_r = kd_tree.resample_nearest(
    swath_cmb,
    data_cmb.surface_precip.data,
    swath_l1c,
    10e3,
    fill_value=np.nan
)
data_cmb  = GPMCMBFile(cmb_file).to_xarray_dataset(smooth=False, profiles=True)

## MRMS data

In [ ]:
from pansat.products.ground_based.mrms import mrms_precip_rate
mrms_files = mrms_precip_rate.download(start_time, start_time)
mrms_data = mrms_precip_rate.open(mrms_files[0])

In [ ]:
from gprof_nn import sensors
from gprof_nn.data.mrms import resample_to_swath
mrms_avg = resample_to_swath(mrms_data, sensors.GMI, l1c_data[{"scans": slice(scan_start, scan_end)}])

In [ ]:
from pyresample import geometry, kd_tree
swath_l1c = geometry.SwathDefinition(
    lats=l1c_data.latitude.data[scan_start:scan_end],
    lons=l1c_data.longitude.data[scan_start:scan_end]
)
swath_mrms = geometry.SwathDefinition(
    lats=mrms_avg.latitude.data,
    lons=mrms_avg.longitude.data
)
sp_mrms_r = kd_tree.resample_nearest(
    swath_mrms,
    mrms_avg.surface_precip_avg.data,
    swath_l1c,
    10e3,
    fill_value=np.nan
)

In [ ]:
from scipy.interpolate import RegularGridInterpolator
from scipy.signal import convolve

lats_mrms = mrms_data.latitude.data
lons_mrms = mrms_data.longitude.data - 360
f = RegularGridInterpolator([lats_mrms[::-1], lons_mrms],
                            mrms_data.precip_rate.data[::-1],
                            bounds_error=False,
                            fill_value=np.nan)
lons, lats = area.get_lonlats()
sp_mrms = f((lats, lons))
sp_mrms[sp_mrms < 0] = np.nan

## Smoothing kernel
#x = np.arange(-20, 21, 2)
#xx, yy = np.meshgrid(x, x)
#r2 = xx ** 2 + yy ** 2
#sigma = (8.5 ** 2) / np.log(0.5)
#k = np.exp(r2 / sigma)
#k /= k.sum()
#
#y = np.nan_to_num(sp_mrms.copy(), 0.0)
#y = convolve(y, k, mode="same")
#y[np.isnan(sp_mrms)] = np.nan
#sp_mrms = y

### Resample GPROF results

In [ ]:
from gprof_nn.data.combined import GPMCMBFile
from pyresample.kd_tree import resample_nearest, resample_gauss
from pyresample.geometry import SwathDefinition, GridDefinition
from scipy.interpolate import RegularGridInterpolator

def resample_surface_precip(data, hr=False):
    lats = data.latitude.data
    lons = data.longitude.data
    swath = SwathDefinition(lats=lats, lons=lons)

    sp = data.surface_precip.data
    sp_r = resample_nearest(swath, sp, area, 15e3, fill_value=np.nan)
    return sp_r

def resample_pixel_status(data):
    lats = data.latitude.data
    lons = data.longitude.data
    swath = SwathDefinition(lats=lats, lons=lons)
    sp = data.pixel_status.data
    sp_r = resample_nearest(swath, sp, area, 15e3, fill_value=-1)
    return sp_r

def resample_ancillary(data):
    lats = data.latitude.data
    lons = data.longitude.data
    swath = SwathDefinition(lats=lats, lons=lons)
    t2m = data.two_meter_temperature.data
    sp = data.surface_type.data
    t2m_r = resample_nearest(swath, t2m, area, 15e3, fill_value=np.nan)
    tcwv = data.total_column_water_vapor.data
    tcwv_r = resample_nearest(swath, tcwv, area, 15e3, fill_value=np.nan)
    sp = data.surface_type.data
    sp_r = resample_nearest(swath, sp, area, 15e3, fill_value=-1)
    return t2m_r, tcwv_r, sp_r

#data_cmb = xr.open_dataset("../data/dorian_cmb_2.nc")
data_gprof = xr.load_dataset("harvey_gprof.nc")[{"scans": slice(scan_start, scan_end)}]
data_nn_1d = xr.load_dataset("harvey_gprof_nn_1d.nc")[{"scans": slice(scan_start, scan_end)}]
data_nn_3d = xr.load_dataset("harvey_gprof_nn_3d.nc")[{"scans": slice(scan_start, scan_end)}]
data_nn_hr = xr.load_dataset("harvey_gprof_hr.nc")[{"scans": slice(scan_start, scan_end)}]
#data_gprof_2d = xr.open_dataset("../data/dorian_gprof_nn_2d.nc")
#sp_cmb = resample_surface_precip(data_cmb)
sp_cmb = resample_surface_precip(data_cmb)
sp_gprof = resample_surface_precip(data_gprof)
sp_gprof_1d = resample_surface_precip(data_nn_1d)
sp_gprof_3d = resample_surface_precip(data_nn_3d)
sp_gprof_hr = resample_surface_precip(data_nn_hr)

sp_gprof[sp_gprof < 0] = np.nan
sp_gprof_1d[sp_gprof < 0] = np.nan
sp_gprof_3d[sp_gprof < 0] = np.nan
#lats_cmb = data_cmb.latitude
#lons_cmb = data_cmb.longitude

In [ ]:
%env CARTOPY_USER_BACKGROUNDS=/home/simon/data/natural_earth/

In [ ]:
import copy
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm, Normalize, ListedColormap
from matplotlib.cm import get_cmap
from matplotlib.patches import Patch
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

import cartopy.crs as ccrs
from PIL import Image

crs = area.to_cartopy_crs()
pc = ccrs.PlateCarree()
f = plt.figure(figsize=(15, 7.3))
gs = GridSpec(2, 4, width_ratios=[1.0, 1.0, 1.0, 0.075])
axs = np.array(
    [[f.add_subplot(gs[i, j], projection=crs) for j in range(3)] for i in range(2)]
)
norm_1 = LogNorm(1e-1, 1e2)


x, y = area.get_proj_coords()
image = background #np.array(Image.open("background.png").convert("RGB"))

mask = sp_gprof < 0
gprof_mask = mask.astype(np.float32)
gprof_mask[~mask] = np.nan

cmap = copy.copy(get_cmap("rocket"))
cmap.set_bad("#00000044")
#cmap_mask = LinearSegmentedColormap.from_list("mask", [(0.0, 1.0, 1.0)], 1)
cmap_mask = ListedColormap(["cyan"], name='from_list', N=None)

levels = np.logspace(-1, 2, 11)

ax = axs[0, 0]
img_extent = crs.x_limits + crs.y_limits
extent = (-800e3, 800e3, -600e3, 600e3)

#ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
#ax.contourf(x, y, sp_mrms, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.pcolormesh(x, y, np.maximum(sp_mrms, 1e-2), norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(a) Ground radar", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=1,
    alpha=0.5,
    linestyle='--'
)
gl.top_labels = False
gl.bottom_labels = False
gl.left_labels = True
gl.right_labels = False
gl.ylocator = mticker.FixedLocator(np.arange(22, 31, 2))
gl.xlines = False
gl.ylines = False

ax = axs[0, 1]
img_extent = crs.x_limits + crs.y_limits

ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, np.maximum(sp_cmb, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(b) GPM Combined", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

#gl.xformatter = LONGITUDE_FORMATTER
#gl.yformatter = LATITUDE_FORMATTER


ax = axs[0, 2]

ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
m = ax.contourf(x, y, np.maximum(sp_gprof, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
ax.contour(x, y, np.isnan(sp_gprof).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(c) GPROF", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")


ax = axs[1, 0]
ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, np.maximum(sp_gprof_1d, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
#ax.pcolormesh(x, y, sp_gprof_1d, norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof_1d).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(d) GPROF-NN 1D", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=1,
    alpha=0.5,
    linestyle='--'
)
gl.top_labels = False
gl.bottom_labels = True
gl.left_labels = True
gl.right_labels = False
gl.xlines = False
gl.ylines = False
gl.ylocator = mticker.FixedLocator(np.arange(22, 31, 4))
gl.xlocator = mticker.FixedLocator(np.arange(-102, 89, 4))


ax = axs[1, 1]
ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
m = ax.contourf(x, y, np.maximum(sp_gprof_3d, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
#m = ax.pcolormesh(x, y, sp_gprof_3d, norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof_3d).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(e) GPROF-NN 3D", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=1,
    alpha=0.5,
    linestyle='--'
)
gl.top_labels = False
gl.bottom_labels = True
gl.left_labels = False
gl.right_labels = False
gl.xlines = False
gl.ylines = False
gl.xlocator = mticker.FixedLocator(np.arange(-102, 89, 4))

ax = axs[1, 2]
ax.set_axis_off()

ax = plt.subplot(gs[:, -1])
plt.colorbar(m, label="Rain rate [mm / h]", cax=ax)

#plt.tight_layout()
plt.show()
f.savefig("surface_precip_harvey_gmi.png", dpi=300)

In [ ]:
import copy
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm, Normalize, ListedColormap
from matplotlib.cm import get_cmap
from matplotlib.patches import Patch
import cartopy.crs as ccrs
from PIL import Image

crs = area.to_cartopy_crs()
pc = ccrs.PlateCarree()
f = plt.figure(figsize=(12, 7.3))
gs = GridSpec(3, 3, height_ratios=[1.0, 1.0, 0.1])
axs = np.array(
    [[f.add_subplot(gs[i, j], projection=crs) for j in range(3)] for i in range(3)]
)
norm_1 = LogNorm(1e-1, 1e2)

extent = (-400e3, 500e3, -600e3, 800e3)

x, y = area.get_proj_coords()
image = background #np.array(Image.open("background.png").convert("RGB"))

mask = sp_gprof < 0
gprof_mask = mask.astype(np.float32)
gprof_mask[~mask] = np.nan

cmap = copy.copy(get_cmap("rocket"))
cmap.set_bad("#00000044")
#cmap_mask = LinearSegmentedColormap.from_list("mask", [(0.0, 1.0, 1.0)], 1)
cmap_mask = ListedColormap(["cyan"], name='from_list', N=None)

levels = np.logspace(-1, 2, 11)

ax = axs[0, 0]
img_extent = crs.x_limits + crs.y_limits
extent = img_extent

ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
#ax.contourf(x, y, sp_mrms, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.pcolormesh(x, y, np.maximum(sp_mrms, 1e-2), norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(a) Ground radar", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[0, 1]
img_extent = crs.x_limits + crs.y_limits
extent = img_extent

ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, np.maximum(sp_cmb, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(b) GPM Combined", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[0, 2]

ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
m = ax.contourf(x, y, np.maximum(sp_gprof, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
ax.contour(x, y, np.isnan(sp_gprof).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(c) GPROF", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[1, 0]
ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, np.maximum(sp_gprof_1d, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
#ax.pcolormesh(x, y, sp_gprof_1d, norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof_1d).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(d) GPROF-NN 1D", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[1, 1]
ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
m = ax.contourf(x, y, np.maximum(sp_gprof_3d, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
#m = ax.pcolormesh(x, y, sp_gprof_3d, norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof_3d).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(e) GPROF-NN 3D", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[1, 2]
ax.background_img("ne_gray")
#ax.imshow(image, origin="upper", extent=img_extent)
m = ax.contourf(x, y, np.maximum(sp_gprof_hr, 1e-4), norm=norm_1, cmap=cmap, levels=levels, extend="both")
#m = ax.pcolormesh(x, y, sp_gprof_3d, norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof_hr).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(e) GPROF-NN HR", loc="left")
#ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="w", lw=1, ls="--", resolution="110m")
ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = plt.subplot(gs[-1, :])
plt.colorbar(m, label="Rain rate [mm / h]", cax=ax, orientation="horizontal")

#plt.tight_layout()
plt.show()
f.savefig("surface_precip_harvey_gmi_hr.png", dpi=300)

In [ ]:
import copy
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm, Normalize, ListedColormap
from matplotlib.cm import get_cmap
from matplotlib.patches import Patch
import cartopy.crs as ccrs
from PIL import Image

crs = area.to_cartopy_crs()
pc = ccrs.PlateCarree()
f = plt.figure(figsize=(12, 7.3))
gs = GridSpec(3, 3, height_ratios=[1.0, 1.0, 0.1])
axs = np.array(
    [[f.add_subplot(gs[i, j], projection=crs) for j in range(3)] for i in range(3)]
)
norm_1 = LogNorm(1e-1, 1e2)

x, y = area.get_proj_coords()
image = background #np.array(Image.open("background.png").convert("RGB"))

mask = sp_gprof < 0
gprof_mask = mask.astype(np.float32)
gprof_mask[~mask] = np.nan

cmap = copy.copy(get_cmap("rocket"))
cmap.set_bad("#00000044")
#cmap_mask = LinearSegmentedColormap.from_list("mask", [(0.0, 1.0, 1.0)], 1)
cmap_mask = ListedColormap(["cyan"], name='from_list', N=None)

levels = np.logspace(-1, 2, 11)

ax = axs[0, 0]
img_extent = crs.x_limits + crs.y_limits
extent = img_extent

#ax.background_img("bm_night")
ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, sp_mrms, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.pcolormesh(x, y, np.maximum(sp_mrms, 1e-2), norm=norm_1, cmap=cmap)
ax.contour(x, y, np.isnan(sp_gprof).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(a) Ground radar", loc="left")
ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[0, 1]
img_extent = crs.x_limits + crs.y_limits
extent = img_extent

#ax.background_img("bm_night")
ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, sp_cmb, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(a) GPM Combined", loc="left")
ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[0, 2]

#ax.background_img("bm_night")
ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, sp_gprof, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.contour(x, y, np.isnan(sp_gprof).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(b) GPROF-NN HR", loc="left")
ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[1, 0]
#ax.background_img("bm_night")
ax.imshow(image, origin="upper", extent=img_extent)
ax.contourf(x, y, sp_gprof_1d, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.contour(x, y, np.isnan(sp_gprof_1d).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(e) GPROF-NN 1D", loc="left")
ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[1, 1]
#ax.background_img("bm_night")
ax.imshow(image, origin="upper", extent=img_extent)
m = ax.contourf(x, y, sp_gprof_3d, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.contour(x, y, np.isnan(sp_gprof_3d).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(f) GPROF-NN 3D", loc="left")
ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[1, 2]
#ax.background_img("bm_night")
ax.imshow(image, origin="upper", extent=img_extent)
m = ax.contourf(x, y, sp_gprof_hr, norm=norm_1, cmap=cmap, levels=levels, extend="max")
ax.contour(x, y, np.isnan(sp_gprof_hr).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5)
ax.contour(x, y, np.isnan(sp_cmb).astype(float), levels=[0.5], colors="aquamarine", linewidths=1.5, linestyles="--")
ax.set_extent(extent, crs=crs)
ax.set_title("(f) GPROF-NN HR", loc="left")
ax.coastlines(color="k", lw=1, ls="-", resolution="110m")
#ax.coastlines(color="white", lw=1, ls=":", resolution="110m")

ax = axs[1, 2]
ax.set_axis_off()

ax = plt.subplot(gs[-1, :2])
plt.colorbar(m, label="Rain rate [mm / h]", cax=ax, orientation="horizontal")


#plt.tight_layout()
plt.show()
f.savefig("surface_precip_harvey_gmi.png", dpi=300)

## Calculate statistics

In [ ]:
import pandas as pd

def get_error_statistics(reference, retrieved, use_pop=False, pop_thresh=0.5):
    sp_ret = retrieved.surface_precip.data
    sp_ref = reference
    valid = (sp_ret >= 0.0) * (sp_ref >= 0.0)
    reference = reference[valid]
    
    sp_ref = sp_ref[valid]
    sp_ret = sp_ret[valid]
    pf = retrieved.precip_flag.data[valid]
    pop = retrieved.pop.data[valid]
    if use_pop:
        print(pop)
        pf = pop > pop_thresh
    
    corr = np.corrcoef(sp_ref, sp_ret)[0, 1]
    
    t = sp_ref > 0
    f = np.isclose(sp_ref, 0)
    p = pf
    tpr = (t * p).sum() / p.sum()
    far = (p * f).sum() / p.sum()
    
    bias = (sp_ret - sp_ref).mean()
    mae = np.abs(sp_ref - sp_ret).mean()
    mse = ((sp_ref - sp_ret) ** 2).mean()
    
    return pd.DataFrame({
        "Biase": [bias],
        "MAE": [mae],
        "MSE": [mse],
        "correlation": [corr],
        "TPR": [tpr],
        "FAR": [far]
    })
    

In [ ]:
data_gprof.surface_precip.shape

In [ ]:
sp_mrms_r.shape

In [ ]:
stats_gprof = get_error_statistics(sp_mrms_r, data_gprof)
stats_nn_1d = get_error_statistics(sp_mrms_r, data_nn_1d)
stats_nn_3d = get_error_statistics(sp_mrms_r, data_nn_3d)

In [ ]:
stats_gprof

In [ ]:
stats_nn_1d

In [ ]:
stats_nn_3d

In [ ]:
stats_gprof = get_error_statistics(sp_cmb_r, data_gprof)
stats_nn_1d = get_error_statistics(sp_cmb_r, data_nn_1d)
stats_nn_3d = get_error_statistics(sp_cmb_r, data_nn_3d)

In [ ]:
bins = np.logspace(-1, 2, 21)

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(15,4 ))

ax = axs[0]
img, _, _ = np.histogram2d(stats_gprof.reference, stats_gprof.retrieved, bins=bins)
ax.pcolormesh(bins, bins, img.T)
ax.set_aspect(1.0)
ax.plot(bins, bins, c="grey", ls="--")
ax.set_xscale("log")
ax.set_yscale("log")

ax = axs[1]
img, _, _ = np.histogram2d(stats_nn_1d.reference, stats_nn_1d.retrieved, bins=bins)
ax.pcolormesh(bins, bins, img.T)
ax.set_aspect(1.0)
ax.plot(bins, bins, c="grey", ls="--")
ax.set_xscale("log")
ax.set_yscale("log")

ax = axs[2]
img, _, _ = np.histogram2d(stats_nn_3d.reference, stats_nn_3d.retrieved, bins=bins)
ax.pcolormesh(bins, bins, img.T)
ax.set_aspect(1.0)
ax.plot(bins, bins, c="grey", ls="--")
ax.set_xscale("log")
ax.set_yscale("log")

plt.show()

### 